In [1]:
import pandas as pd
import numpy as np
import os
os.environ['MKL_NUM_THREADS'] = '1'
from collections import Counter
import itertools
import json
import scipy.sparse as sparse
import pickle
import torch
import misc.util as util
from torch import nn

import importlib
from torch.utils.data import DataLoader
from misc.loader import AEDataset
from misc.util import *
from tqdm.auto import tqdm
from eval.rec_eval import *
import neuralsort.pl as pl
from models.loss import neuPrecLoss
from misc.loader import RecDataset
import models
from models.loss import *
from sklearn.utils import shuffle

In [2]:
dataset_name = 'ml-1m-l-1-100'

In [3]:
import pickle
with open("data/parsed/%s" % dataset_name, 'rb') as f:
    tr, val = pickle.load(f)

In [4]:
tr.shape

(6034, 2811)

In [5]:
import models.mf
import importlib
import misc.loader
importlib.reload(models.mf)
importlib.reload(misc.loader)

<module 'misc.loader' from '/Users/ita/Dropbox/code/neurank/code/misc/loader.py'>

In [6]:
dset = misc.loader.RecDataset(tr)
loader = DataLoader(dset, batch_size=2048, shuffle=True, num_workers=8)
for i, x in enumerate(loader):
    u, i, j = x
    break

In [7]:
model = models.mf.mfrec(*tr.shape)

In [8]:
r = model(u, i, j)

In [9]:
import models.neuralsort
importlib.reload(models.neuralsort)

<module 'models.neuralsort' from '/Users/ita/Dropbox/code/neurank/code/models/neuralsort.py'>

In [10]:
val[0]

array([  36,  326, 1367,  154,  108, 1329, 2741, 2491, 1677,  258, 1184,
       2520,  112, 2416, 1995,  859, 2111, 2304,   42, 1973,  652, 1561,
        925, 2341,  563, 2604, 2203,  945, 1962, 1804, 2625,  789, 1337,
       2003, 1984, 2591, 1239,  996,  732, 1362, 1301,   29,  409,  922,
       2709,  297,  119, 1669,  202, 1433, 1933, 2635, 2056,  269,  112,
       2254,  114,  611, 2127,  773,  935, 1948,  597,  180, 2203,   53,
        402, 2358,  115, 2220, 1534,  737, 2742, 2205, 1978,  614, 1580,
       1196, 1493,  944, 2778, 2597, 1253, 1902,  489, 1172, 1820, 1924,
       1479,  381, 1942,  996,  617, 2403,  353, 1393,  485, 1518, 2374,
       2539], dtype=int32)

In [11]:
model.predict(0, val[0])

array([-0.33695596, -0.31838104, -0.28122538, -0.34366965, -0.23285285,
       -0.3109648 , -0.32853332, -0.3289242 , -0.22991976, -0.37208927,
       -0.24132055, -0.2941813 , -0.32621798, -0.44292116, -0.28945875,
       -0.3362897 , -0.30960107, -0.21832374, -0.19975227, -0.36006117,
       -0.28442687, -0.4129879 , -0.36073166, -0.32514206, -0.25009775,
       -0.34351632, -0.25892526, -0.17259684, -0.3570848 , -0.3251905 ,
       -0.2010753 , -0.22881794, -0.35180494, -0.32921216, -0.3298677 ,
       -0.2722623 , -0.33458394, -0.31946203, -0.28045815, -0.21363012,
       -0.44525456, -0.22865492, -0.33605614, -0.3543511 , -0.23016697,
       -0.41400906, -0.1989606 , -0.2580688 , -0.31335127, -0.38501006,
       -0.3040734 , -0.32426727, -0.14592464, -0.24491836, -0.32621798,
       -0.33035368, -0.26311517, -0.3190806 , -0.27301794, -0.28437895,
       -0.37871084, -0.2682964 , -0.30563515, -0.38559058, -0.25892526,
       -0.33617312, -0.2958508 , -0.28746572, -0.2578489 , -0.25

In [12]:
leave_k_eval(model, tr, val, leavek=1, K=5)

{'precision': 0.047232350016572756,
 'map': 0.02204452546679925,
 'ndcg': 0.028193001931852795}

In [13]:
model.predict(3, val[10])

array([-0.32316136, -0.35620803, -0.31305772, -0.25639752, -0.33693746,
       -0.42032105, -0.33492276, -0.31180927, -0.37709266, -0.45499116,
       -0.3487684 , -0.40809727, -0.46335837, -0.39495432, -0.2388192 ,
       -0.25863945, -0.22911467, -0.37999   , -0.3396332 , -0.2388192 ,
       -0.26304066, -0.399331  , -0.28320637, -0.25945947, -0.4592467 ,
       -0.3045873 , -0.42373726, -0.3346524 , -0.3717785 , -0.29744682,
       -0.31219342, -0.32798487, -0.41286182, -0.28778163, -0.30139583,
       -0.3174459 , -0.3498651 , -0.32392594, -0.34584296, -0.29266137,
       -0.2782405 , -0.36220303, -0.36714494, -0.34518278, -0.32607743,
       -0.33713993, -0.43702593, -0.41253713, -0.35926   , -0.29493457,
       -0.37164947, -0.2430844 , -0.38780895, -0.33432162, -0.40103608,
       -0.38433492, -0.39328846, -0.33885998, -0.30833665, -0.34052247,
       -0.46606338, -0.2596262 , -0.26589465, -0.37664706, -0.22206849,
       -0.39169   , -0.4632836 , -0.44875202, -0.4304592 , -0.37